# Technical note: automatic differentiation

**This is an older document, describing autodiff functionality that will probably be deprecated soon**

Some Myokit functions use a special `Differential` data type to numerically evaluate (partial) derivatives.
These are described below.


## JacobianGenerator

We start from a system

\begin{equation}
\dot{y}\left(t\right)=f\left(y\left(t\right),u\left(t\right),t|p\right)
\end{equation}

For the remainder of this section, we assume $p$ is uninteresting and drop it from the notation.
The JacobianGenerator class starts from a log containing log entries that contain the basic info

\begin{equation}
\left\langle y\left(t\right),u\left(t\right),t\right\rangle
\end{equation}

for each logged point.
Next, it revisits each point and makes a call to the RHS function.
However, instead of running it with floats or doubles, it uses a specialised datatype called `Differential`.
This data type contains both a scalar value and a list of partial derivatives.
All operations are overloaded so that a calculation

\begin{equation}
x\left(s\right)=a\left(s\right)\cdot b\left(s\right)
\end{equation}

is augmented with an operation

\begin{equation}
\frac{\partial x}{\partial s}=\frac{\partial a}{\partial s}b+\frac{\partial b}{\partial s}a
\end{equation}

for each partial derivative with respect to $s$ stored in the `Differential` type.
This allows a derivative to be calculated numerically from the same equations used to calculate the original value.

In the `JacobianGenerator`, each `Differential` is defined to contain $n_{state}$ partial derivatives. 
Before each call to the RHS, each state variable $y_{i}$ is initialised to contain $n_{state}-1$ zeros, and a single $1$ at position $i$ (because $\frac{\partial y_{i}}{\partial y_{i}}=1$).
All constants are introduced as differentials with $n_{state}$ zeros and intermediary variables obtain their partial derivatives from the states they interact with. 
After a call to the RHS function, the calculated derivatives are returned as `Differential` objects containing a pair
$\left\langle f_{i},\frac{\partial f_{i}}{\partial y_{j}}\right\rangle $.
Combining the partial derivatives stored in every returned derivative variable, we obtain the full Jacobian matrix for time $t$.


### Interpreation

TODO: Eigenvalues, dominant eigenvalue



## ICSimulation

The ICSimulation uses the same technique as the JacobianGenerator, but with two crucial differences:

1. At the very start of the simulation, all states are initialised with $n_{state}-1$ zeros, and a single $1$ at position $i$.
   For each subsequent call to the RHS function, the previous state $y$ is used without reinitializing.
- The state variables are integrated at each step, leading not only to the value of $y$ at each time $t$, but also the partial derivatives of $y$ with respect to $y\left(t=t_{min}\right)$.

### Interpretation

To interpret these results, it's useful to introduce the _flow function:_

\begin{equation}
\phi\left(y_{0},t\right)\rightarrow y\left(t\right)
\end{equation}

For any initial position $y_{0}$ and interval $t$ this function gives us the value of $y$ after $t$ time units.
In effect, an ODE simulation is a technique to evaluate $\phi$ for a given $y_{0}$ and $t$.

With this definition, we can easily write down what the `ICSimulation` does:
For any pair $\left\langle y_{0},t\right\rangle $ it returns a pair $\left\langle y(t),\left.\frac{\partial y}{\partial y_{o}}\right|_{t}\right\rangle $.

TODO: Eigenvalues, dominant eigenvalue

## PSimulation


See Dickinson and Gelinas, 1975, Sensitivity analysis of ordinary differential equation systems -- a direct method.

Initially, $y$ and $p$ are scalars:

\begin{align}
\frac{dy}{dt} &= f(y(t, p), t, p) \\
y(t = t_0)    &= y_0(p)
\end{align}

Sensitivity

\begin{align}
s = \frac{\partial y}{\partial p}
\end{align}

Now look at the time-derivative of $s$:

\begin{align}
\frac{d}{d t} (s)
    = \frac{d}{d t} \left( \frac{\partial}{\partial p} y \right)
    = \frac{\partial}{\partial p} \left( \frac{d y}{d t} \right)
\end{align}
or
\begin{align}
\frac{d s}{d t} = \frac{\partial}{\partial p} f(y(t, p), t, p)
\end{align}

Here $f$ is a function of $p$ in two ways: via $y(t, p)$ and via $p$, so we need to treat both as functions of $p$ and use the chain rule for multivariate functions:

\begin{align}
\frac{\partial}{\partial p} f(y(t, p), t, p) &=
    \frac{\partial f}{\partial y}\frac{\partial y}{\partial p}
    + \frac{\partial f}{\partial p}\frac{\partial p}{\partial p} \\
&= \frac{\partial f}{\partial p} + \frac{\partial f}{\partial y}\frac{\partial y}{\partial p} \\
&= \frac{\partial f}{\partial p} + \frac{\partial f}{\partial y} s
\end{align}

For a vector $y$, we have to add a term for each of $y$'s components.

Let
\begin{align}
\frac{dy_i}{dt} &= f_i(y_1, y_2, ..., y_n, t, p), \qquad i = 1,...,n \\
s_i &= \frac{\partial y_i}{\partial p}
\end{align}
then
\begin{align}
\frac{\partial}{\partial p} f(y(t, p), t, p)
    = \frac{\partial f}{\partial p} + \sum_{j=1}^n \frac{\partial f_i}{\partial y_j} s_j
\end{align}

Myokit's `PSimulation` uses forward Euler to approximate $y$, based on $f$.
But instead of using doubles, it uses a `Differential` type, which uses automatic differentiation to calculate selected derivatives while it does so.

Lets say we investigate a single parameter $p$, then every state will be represented internally as
\begin{align}
\left( y[t], \frac{dy[t]}{dp} \right)
\end{align}
while a current will be
\begin{align}
\left( I[t], \frac{dI[t]}{dp} \right)
\end{align}
and the parameter itself:
\begin{align}
\left( p, \frac{dp}{dp} \right) = \left( p, 1 \right)
\end{align}

At the start of the simulation we set all derivative components to 0, except for those of the parameters.
Next, we perform a single RHS evaluation, to obtain
\begin{align}
\left( f[0], \frac{df[0]}{dp} \right)
\end{align}
(at this point, the 1 from $dp/dp$ will have propagated into $df[0]/dp$).

Note that we know have $\frac{df}{dp}$, not $\frac{\partial f}{\partial p}$, which is different (see above).
We use Euler to integrate, and obtain
\begin{align}
\left( y[0], \frac{dy[0]}{dp} \right)
\end{align}
And then we can continue this to find $dy/dp$ at any point in time.

(However, it's possible that the errors from each step add up)